# Using pretrained MobileNetV2

Here we want to use a pre-trained mobile net and train it on the hand images from before. If we use the model already implemented in keras, we can use the weights from ImageNet.

# importing libraries

In [47]:
import tensorflow as tf
from tensorflow import keras

from tensorflow.keras import backend as K
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.layers import Dropout

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image

from tensorflow.keras.models import Model
#from tensorflow.keras.applications import imagenet_utils
from tensorflow.keras.layers import Dense,GlobalAveragePooling2D
from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2

from PIL import Image # used for loading images
from tensorflow.keras.applications.mobilenet import preprocess_input
import numpy as np
#from IPython.display import Image
from tensorflow.keras.optimizers import Adam

import matplotlib.pyplot as plt


In [48]:
print(tf.version)

<module 'tensorflow._api.v2.version' from 'C:\\Users\\werth\\AppData\\Local\\Continuum\\anaconda3\\envs\\workshop\\lib\\site-packages\\tensorflow\\_api\\v2\\version\\__init__.py'>


# The model
## Loading the mobileNet model without top

In [49]:
base_model=MobileNetV2(weights='imagenet',include_top=False) #imports the mobilenet model and discards the last 1000 neuron layer.

x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(1024,activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
x=Dense(1024,activation='relu')(x) #dense layer 2
x=Dense(512,activation='relu')(x) #dense layer 3

preds=Dense(2,activation='softmax')(x) #final layer with softmax activation

#specify the inputs
#specify the outputs
model=Model(inputs=base_model.input,outputs=preds)
#now a model has been created based on our architecture

C:\Users\werth\AppData\Local\Continuum\anaconda3\envs\workshop\lib\site-packages\keras_applications\mobilenet_v2.py:295: UserWarning: MobileNet shape is undefined. Weights for input shape(224, 224) will be loaded.
  warnings.warn('MobileNet shape is undefined.'


## visualize models to compare with and without top

In [50]:
for i,layer in enumerate(model.layers):
   # print(i,layer.name)
    1

Just for comparison, the original mobileNetV2 below

In [51]:
compare_model=MobileNetV2(weights='imagenet',include_top=True) #imports the mobilenet model and discards the last 1000 neuron layer.
for i,layer in enumerate(compare_model.layers):
    #print(i,layer.name)
    1

In [ ]:
from keras.utils import plot_model
plot_model(model, to_file='model.png')

## What happens when removing the last dense layers (on top)?

the weights in a convolutional layer are fixed-size. They are the size of the kernel x filters. Example: a 3x3 kernel of 10 filters. A convolutional layer doesn't care about the size of the input image. It just does the convolutions and present a resulting image based on the size of the input image. (Search for some illustrated tutorials about convolutions if this is unclear)

now the weights in a dense layer are totally dependent on the input size. It's one weight per element of the input. So this demands that your input be always the same size, or else you won't have proper learned weights.

Because of this, removing the final dense layers allows you to define the input size (see in documentation). (And the output size will increase/decrease accordingly)

Thanks to great stackoverflower [Daniel Möller](https://stackoverflow.com/users/2097240/daniel-m%c3%b6ller)

## Fixing the layers to not be trainable

The model is set up like a numpy array where you can reach each layer by itself. You can set now all layers of the base model frozen.
important is that after setting layer trainable =False, the model has to `be compile()` to take effect.


See Keras [webside](https://keras.io/getting-started/faq/#how-can-i-freeze-keras-layers)

In [52]:
n=100 #149 are all layers before dense output
for layer in model.layers[:n]:
    layer.trainable=False
for layer in model.layers[n:]:
    layer.trainable=True

In [53]:
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['categorical_accuracy'])



For optimal throughput I used a double-buffering approach where the next request is already being prepared (by the CPU) while the current one is still being processed (by the GPU). This way the CPU and GPU are never waiting for one another.

(Fun fact: for V2 it was actually worth doing triple buffering but for V1 that made no difference in speed. This shows that V2 is much more efficient.)

---

# Training model
## Loading the data with data generator

Now we use the generator function again to load the data for training

In [54]:
datagen =ImageDataGenerator()

train_batch_generator =datagen.flow_from_directory(directory="C:/Users/werth/Pictures/small_structured/input/Train/",
                                     classes=['Faust', 'Offen'],
                                     target_size=[170,255],
                                     class_mode='categorical',
                                     batch_size=8,
                                     color_mode='rgb',
                                     shuffle=True,
                                     seed=42)

valid_batch_generator =datagen.flow_from_directory(directory="C:/Users/werth/Pictures/small_structured/input/Test/",
                                     classes=['Faust', 'Offen'],
                                     target_size=[170,255],
                                     class_mode='categorical',
                                     batch_size=5,
                                     color_mode='rgb',
                                     shuffle=True,
                                     seed=42)

#img_batch = next(batches)



Found 264 images belonging to 2 classes.
Found 10 images belonging to 2 classes.


## Training the model

In [62]:
%%time
step_size_train=train_batch_generator.n//train_batch_generator.batch_size
step_size_valid=valid_batch_generator.n//valid_batch_generator.batch_size

history=model.fit_generator(generator=train_batch_generator,
                    steps_per_epoch=step_size_train,
                    validation_data=valid_batch_generator,
                    epochs=5,
                    callbacks=[tensorboard_callback])

Epoch 1/5
33/33 [==============================] - 97s 3s/step - loss: 0.3697 - categorical_accuracy: 0.8447 - val_loss: 11.7013 - val_categorical_accuracy: 0.3000
Epoch 2/5
33/33 [==============================] - 34s 1s/step - loss: 0.2920 - categorical_accuracy: 0.9394 - val_loss: 3.5547 - val_categorical_accuracy: 0.3000
Epoch 3/5
33/33 [==============================] - 30s 904ms/step - loss: 0.1748 - categorical_accuracy: 0.9545 - val_loss: 17.3694 - val_categorical_accuracy: 0.3000
Epoch 4/5
33/33 [==============================] - 28s 852ms/step - loss: 0.3986 - categorical_accuracy: 0.9318 - val_loss: 23.5045 - val_categorical_accuracy: 0.3000
Epoch 5/5
33/33 [==============================] - 28s 851ms/step - loss: 0.0212 - categorical_accuracy: 0.9924 - val_loss: 9.9980 - val_categorical_accuracy: 0.4000
Wall time: 3min 44s


# Evaluating the model 

## Tensorboard

before training the model we activate tensorboard. Tensorboard can monitor the training and validation of the model and visualize the process. Thereby we can identify where optimization is needed.

In [69]:
import datetime, os
%load_ext tensorboard.notebook

logs_base_dir = ".\logs"#create folder for 
os.makedirs(logs_base_dir, exist_ok=True)
%tensorboard --logdir {logs_base_dir}

logdir = os.path.join(logs_base_dir, datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

Alternatively you can also use a file writer and save the information in the folder. Than you can call the tensorboard via the console and open tensorboard in another tab

In [65]:
writer = tf.summary.FileWriter( "./logs", tf.get_default_graph())
%tensorboard --logdir=.\logs

ERROR: Timed out waiting for TensorBoard to start. It may still be running as pid 12504.

# Simply printing the history

In [ ]:
print(history.history.keys())

In [ ]:
# summarize history for accuracy
plt.plot(history.history['categorical_accuracy'])
plt.plot(history.history['val_categorical_accuracy'])
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()